In [26]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lmbmo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
df = pd.read_csv("data.csv")

In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
df.shape

(20800, 5)

In [6]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
df = df.dropna()
df.shape

(18285, 5)

In [8]:
# merging the author name and news title
df['content'] = df['author']+' '+df['title']

In [9]:
df['content'].head()

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: content, dtype: object

In [10]:
# getting the independent features
X = df.drop('label', axis=1)
# getting the dependent feature
y = df['label']

print(X.shape)
print(y.shape)

(18285, 5)
(18285,)


In [11]:
messages=X.copy()
messages['content'][1]

'Daniel J. Flynn FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [12]:
messages.reset_index(inplace=True)
messages

,index,id,title,author,text,content
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Consortiumnews.com Why the Truth Might Get You...
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Howard Portnoy Iranian woman jailed for fictio...
...,...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,"Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma..."
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,Michael J. de la Merced and Rachel Abrams Macy...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...","Alex Ansary NATO, Russia To Hold Parallel Exer..."


### Preprocessing and stemming

In [13]:
# Dataset Preprocessing
ps = PorterStemmer()
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['content'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [14]:
corpus[1:10]

['daniel j flynn flynn hillari clinton big woman campu breitbart',
 'consortiumnew com truth might get fire',
 'jessica purkiss civilian kill singl us airstrik identifi',
 'howard portnoy iranian woman jail fiction unpublish stori woman stone death adulteri',
 'daniel nussbaum jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'alissa j rubin beno hamon win french socialist parti presidenti nomin new york time',
 'megan twohey scott shane back channel plan ukrain russia courtesi trump associ new york time',
 'aaron klein obama organ action partner soro link indivis disrupt trump agenda',
 'chri tomlinson bbc comedi sketch real housew isi caus outrag']

In [15]:
## one hot representation
voc_size=10000
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1:3]

[[5350, 4884, 4930, 4930, 6603, 2831, 6150, 5073, 3146, 6232],
 [9719, 9584, 8994, 5063, 8141, 2898]]

In [25]:
print(corpus[0])
print(onehot_repr[0])

darrel lucu hous dem aid even see comey letter jason chaffetz tweet
[1970, 432, 5482, 9450, 6829, 2055, 8333, 7957, 4426, 4815, 652, 4544]


### Embedding representation

In [18]:
sent_length=25
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1970  432 5482 ...    0    0    0]
 [5350 4884 4930 ...    0    0    0]
 [9719 9584 8994 ...    0    0    0]
 ...
 [1912 4884 7669 ...    0    0    0]
 [9500 2281 5420 ...    0    0    0]
 [6625  833 8830 ...    0    0    0]]


### Creating model

In [27]:
## creating model
embedded_vector_features=40#number of features
model=Sequential()
model.add(Embedding(voc_size,embedded_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))#number of nuerons
model.add(Dense(1,activation='sigmoid'))# as it is binary classifier
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 40)            400000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [29]:
X_final.shape,y_final.shape

((18285, 25), (18285,))

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=33)

In [31]:
##model training
model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=40, batch_size=32)

Epoch 1/40
400/400 [==============================] - 20s 35ms/step - loss: 0.1411 - accuracy: 0.9352 - val_loss: 0.0366 - val_accuracy: 0.9896
Epoch 2/40
400/400 [==============================] - 13s 34ms/step - loss: 0.0143 - accuracy: 0.9955 - val_loss: 0.0339 - val_accuracy: 0.9876
Epoch 3/40
400/400 [==============================] - 12s 30ms/step - loss: 0.0073 - accuracy: 0.9979 - val_loss: 0.0362 - val_accuracy: 0.9894
Epoch 4/40
400/400 [==============================] - 12s 30ms/step - loss: 0.0019 - accuracy: 0.9995 - val_loss: 0.0436 - val_accuracy: 0.9892
Epoch 5/40
400/400 [==============================] - 12s 30ms/step - loss: 0.0111 - accuracy: 0.9974 - val_loss: 0.0557 - val_accuracy: 0.9820
Epoch 6/40
400/400 [==============================] - 12s 29ms/step - loss: 0.0053 - accuracy: 0.9987 - val_loss: 0.0436 - val_accuracy: 0.9871
Epoch 7/40
400/400 [==============================] - 12s 30ms/step - loss: 0.0040 - accuracy: 0.9985 - val_loss: 0.0866 - val_accuracy:

In [32]:
y_pred=model.predict(X_test)

172/172 [==============================] - 3s 8ms/step


In [33]:
y_pred=np.where(y_pred>=0.5,1,0)

In [34]:
conf_mat = confusion_matrix(y_test,y_pred)
acc = accuracy_score(y_test,y_pred)
class_rep = classification_report(y_test,y_pred)

print(conf_mat)
print(f'Accuracy is {acc}')
print(class_rep)

[[3107   49]
 [  87 2243]]
Accuracy is 0.9752096244987241
              precision    recall  f1-score   support

           0       0.97      0.98      0.98      3156
           1       0.98      0.96      0.97      2330

    accuracy                           0.98      5486
   macro avg       0.98      0.97      0.97      5486
weighted avg       0.98      0.98      0.98      5486

